In [ ]:
import math
import logging
import numpy as np
import pandas as pd
import scipy.signal as sig
from scipy.stats import linregress
from scipy.interpolate import InterpolatedUnivariateSpline
import altair as alt
import flammkuchen as fl
from ray import tune
from ray.tune import ProgressReporter, JupyterNotebookReporter

import sys,os,os.path
sys.path.append(os.path.expanduser('../src'))

from spinorama.filter_iir import Biquad
from spinorama.filter_peq import peq_build, peq_print
from spinorama.load import graph_melt
from spinorama.load_rewseq import parse_eq_iir_rews
from spinorama.graph import graph_spinorama, graph_freq
from spinorama.compute_scores import scores
from spinorama.filter_scores import scores_apply_filter, scores_print, scores_loss

In [ ]:
df_all_speakers = fl.load('../cache.parse_all_speakers.h5')

In [ ]:
df_speaker = df_all_speakers['KEF LS50']['ASR']['asr']
df_speaker.keys()
original_mean = df_speaker['CEA2034_original_mean']

In [ ]:
g_params = {'xmin': 20, 'xmax': 20000, 'ymin': -50, 'ymax': 10, 'width': 400, 'height': 250}
score = scores(df_speaker)
# print(score)
# print(score['pref_score'])

In [ ]:
df = df_speaker['CEA2034_unmelted'].loc[:,{'Freq', 'Listening Window'}]
freq = df.loc[df['Freq']>300,'Freq'].values
lw = df.loc[df['Freq']>300,'Listening Window'].values
slope, intercept, r_value, p_value, std_err = linregress(freq,lw)

global_target = lw-[(slope*freq[i]+intercept) for i in range(0, len(freq))]
global_peq = []

for i in range(0, 10):

    # currently a line between 300Hz and 20kHz
    target = global_target+peq_build(freq, global_peq)
    
    # greedy strategy: look for highest peak
    n = 1
    indice_max = np.argpartition(target, n-1)[:n]
    init_freq = freq[indice_max[0]]
# OOPS    if init_freq in getout
    init_freq_min = max(init_freq*0.9, 20)
    init_freq_max = min(init_freq*1.1, 20000)
    init_freq_range = np.linspace(init_freq_min,  init_freq_max, 5).tolist()
    print('Debug: freq min {}Hz peak {}Hz max {}Hz'.format(init_freq_min, init_freq, init_freq_max))
    
    # estimate area
    spline = InterpolatedUnivariateSpline(np.log10(freq), target, k=1)
    init_dbGain = spline(np.log10(indice_max))[0]
    init_dbGain_min = -12 # max(init_dbGain/5, 0.5)
    init_dbGain_max = 12  # min(init_dbGain+2, 12)
    init_dbGain_range = np.linspace(init_dbGain_min, init_dbGain_max, 50).tolist()
    print('Debug: gain min {}dB peak {}dB max {}dB'.format(init_dbGain_min, init_dbGain, init_dbGain_max))
    
    # range for Q
    init_Q_range = np.linspace(1, 5, 10).tolist()

    
    def lw_loss(delta, peq):
        return np.linalg.norm(delta+peq_build(freq, peq))

    def lw_optimizer(config, checkpoint_dir = '/var/vm/checkpoints'):
        current_peq = [
            (1.0, Biquad(Biquad.PEAK, config['1_freq'], 48000, config['1_Q'], config['1_dbGain'])),
        ]
        intermediate_score = lw_loss(target, current_peq)
        tune.report(mean_loss=intermediate_score)

    lw_analysis = tune.run(
        lw_optimizer,
        config={
            "1_freq": tune.grid_search(init_freq_range),
            "1_Q": tune.grid_search(init_Q_range),
            "1_dbGain": tune.grid_search(init_dbGain_range),
        },
        progress_reporter=JupyterNotebookReporter(overwrite=True, max_progress_rows=5, max_report_frequency=30, print_intermediate_tables=False),
        resources_per_trial={"cpu": 1},
        metric='mean_loss',
        mode='min')
    
    best = lw_analysis.get_best_trial(metric='mean_loss',mode='min').last_result
    print(best)
    global_peq.append((1.0, 
                       Biquad(Biquad.PEAK, 
                              best['config']['1_freq'],
                              48000,
                              best['config']['1_Q'],
                              best['config']['1_dbGain'])))


print(global_peq)

In [ ]:
peq_print(global_peq)

In [ ]:
peq_print(my_peq)

In [ ]:
my_fs = 48000
my_peq = parse_eq_iir_rews('../datas/eq/KEF LS50/iir.txt', my_fs)
spin_filtered, pir_filtered, score_filtered = scores_apply_filter(df_speaker, my_peq)
spin_auto, pir_auto, score_auto = scores_apply_filter(df_speaker, global_peq)
scores_print(score, score_filtered)
scores_print(score, score_auto)

In [ ]:
graph_spinorama(df_speaker['CEA2034'], g_params) & graph_spinorama(spin_filtered, g_params) & graph_spinorama(spin_auto, g_params)